In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
um=1e-6
mm=1e-3
nm=1e-9


In [2]:
def mask_cos(n,m,dx,dy,m1,l):
    x=np.arange(-int(m/2),int(n/2),1)
    y=np.arange(-int(n/2),int(m/2),1)
    
    X,Y=np.meshgrid(x,y)
    X=X*(dx)
    Y=Y*(dy)
    mask= 0.5*(1+m1*np.cos((2*np.pi*X)/l))
    return mask


    
    
    
    


In [3]:
%matplotlib qt

In [4]:
mask1=mask_cos(1024,1024,600*nm,600*nm,1,32*um)
#mask1=np.pad(mask1, ((1024, 1024), (1024, 1024)), 'constant', constant_values=0)
plt.figure()
plt.imshow(mask1)
plt.show()
dx=600*nm
dy=600*nm
w=632.8*nm
z1=500*um

In [5]:
def d_angular( U,dx,dy,z,W_length):
    freq_U=np.fft.fftshift(np.fft.fft2(U)) 
    N,M =np.shape(U)
    x=np.arange(-int(M/2),int(M/2),1)
    y=np.arange(-int(N/2),int(N/2),1)
    
    X,Y=np.meshgrid(x,y)
    k=(2*np.pi)/W_length
    dfx=X*1/(M*dx)
    dfy=Y*1/(N*dy)
    
    c1=1j*z*k
    matrix= np.exp(c1*np.sqrt(1-((W_length**2)*(dfx**2+dfy**2))))
    matrix2=freq_U*matrix
    matrix3=np.fft.ifft2(matrix2)
    
    return matrix3
    


In [6]:
def T_fresnel( U,dx,dy,z,W_length):
    N,M =np.shape(U)
    x=np.arange(-int(M/2),int(M/2),1)
    y=np.arange(-int(N/2),int(N/2),1)
    
    X,Y=np.meshgrid(x,y)
    k=(2*np.pi)/W_length
    dx0=X*(dx)
    dy0=Y*(dy)
    
    c1=(1j*k)/(2*z)
    matrix= np.exp(c1*(dx0**2+dy0**2))
    matrix2=U*matrix
    matrix3=np.fft.fftshift(np.fft.fft2(matrix2))
    
    x1=np.arange(-int(M/2),int(M/2),1)
    y1=np.arange(-int(N/2),int(N/2),1)
    X1,Y1=np.meshgrid(x1,y1)
    dx1=X1*((W_length*z)/(M*dx))
    dy1=Y1*((W_length*z)/(N*dy))
    
    c2=np.exp(1j*k*z)
    c3=1j*W_length*z
    c4=c2/c3
    matrix4= c4*np.exp(c1*(dx1**2+dy1**2))
    U_final=matrix3*matrix4
    
    
    return U_final

In [7]:
def z_critic(m,dx,w):
    zcr=(m*(dx**2))/w
    return zcr

In [8]:
zcr1=z_critic(2048,dx,w)
print(zcr1)

0.001165107458912769


In [9]:
z_l=((32*um)**2)/(632.8*nm)

In [10]:
img55=T_fresnel(mask1,dx,dy,3*z_l,w)
plt.figure()
plt.imshow((np.abs(img55)**2))
plt.colorbar()
plt.show()

In [11]:
z_l

0.001618204804045512

In [12]:
img56=T_fresnel(mask1,dx,dy,2*z_l,w)
plt.figure()
plt.imshow((np.abs(img56)**2))
plt.colorbar()
plt.show()

In [13]:
def mask_concentric(n,m,r2,r1):
    mascara=np.zeros((n,m))
    cv.circle(mascara,(int(n/2), int(m/2)),r2,1,-1)
    cv.circle(mascara,(int(n/2), int(m/2)),r1,0,-1)
    return mascara

In [14]:
con_mask=mask_concentric(2048,2048,800,300)
plt.figure()
plt.imshow(con_mask)
plt.colorbar()
plt.show()

In [15]:
def fresnel_zones_circular2(r2,r1,z,w_length):
    zones=((r1)**2)/(z*w_length)
    return zones

In [16]:
def Zfresnel_zones_circular2(r2,r1,N,w_length):
    zeta=((r1)**2)/(N*w_length)
    return zeta

In [17]:
fresnel_zones_circular2(800*dx,300*dx,5*mm,w)

10.240202275600506

In [18]:
z_c=Zfresnel_zones_circular2(500*dx,300*dx,15,w)
z_c

0.0034134007585335025

In [19]:
img56=T_fresnel(con_mask,dx,dy,z_c,w)
#img56=(img56-np.min(img56))/(np.max(img56)-np.min(img56))
plt.figure()
plt.imshow((np.abs(img56)))
plt.colorbar()
plt.show()